<h1>How the backpropagation algorithm works</h1>

In this chapter we'll explain a fast algorithm for computing such gradients, an algorithm known as backpropagation.

At the heart of backpropagation is an expression for the partial derivative $\frac{\partial C}{\partial w}$ of the cost function $C$ with respect to any weight $w$ (or bias $b$) in the network. The expression tells us how quickly the cost changes when we change the weights and biases.

<h2>A fast matrix-based approach to computing the output from a neural network</h2>

Let's warm up with a fast matrix-based algorithm to compute the output from a neural network. We'll use $w^l_{jk}$ to denote the weight for the connection from the $k^{th}$ neuron in the $(l-1)^{\rm th}$ layer to the $j^{th}$ neuron in the $l^{th}$ layer.

![Figure 2.1](imgs/network1.png)

We use $b^l_j$ for the bias of the $j^{th}$ neuron in the $l^{th}$ layer. And we use $a^l_j$ for the activation of the $j^{th}$ neuron in the $l^{th}$ layer. For example,

![Figure 2.2](imgs/network2.png)

The activation $a^l_j$of the $j^{th}$ neuron in the $l^{th}$ layer is related to the activations in the $(l-1)^{\rm th}$ layer by the equation

$$
\begin{eqnarray} 
  a^{l}_j = \sigma\left( \sum_k w^{l}_{jk} a^{l-1}_k + b^l_j \right),
\tag{23}\end{eqnarray}
$$

over all neurons $k$ in the $(l-1)^{\rm th}$ layer

To rewrite this expression in a matrix form we define a weight matrix $w^l$ for each layer, $l$. The entries of the weight matrix $w^l$ are just the weights connecting to the $l^{th}$ layer of neurons, that is, the entry in the $j^{th}$ row and $k^{th}$ column is $w^l_{jk}$. Similarly, for each layer $l$ we define a bias vector, $b^l$. And finally, we define an activation vector $a^l$ whose components are the activations $a^l_j$.

The last ingredient we need to rewrite (23) in a matrix form is the idea of vectorizing a function such as $\sigma$. We want to apply a function such as σ to every element in a vector $v$. We use the obvious notation $\sigma(v)$ to denote this kind of elementwise

So, equation (23) can be written as

$$
\begin{eqnarray} 
  a^{l} = \sigma(w^l a^{l-1}+b^l).
\tag{25}\end{eqnarray}
$$

Before we apply the sigmoid function, we comput the intermediate quantity $z^l \equiv w^l a^{l-1}+b^l$. We call this the *weighted input* to the neurons in layer $l$.

Equation (25) is sometimes written as $a^l = \sigma(z^l)$

<h2>The two assumptions we need about the cost function</h2>

The goal of backpropagation is to compute the partial derivatives $\partial C / \partial w$ and $\partial C / \partial b$ of the cost function $C$ with respect to any weight w or bias b in the network.

*$\partial C / \partial w$ can be interpreted as the instantaneous change of $C$ with respect to the weights. That is, we want to find the value of weights that make C the smallest.*

We need to make two main assumptions about the cost function for backpropagation to work. We will use the quadratic cost function as an example in the form:

$$
\begin{eqnarray}
  C = \frac{1}{2n} \sum_x \|y(x)-a^L(x)\|^2,
\tag{26}\end{eqnarray}
$$

* $n$ is the total number of training examples <br>
* the sum is over individual training examples, $x$ <br>
* $y=y(x)$ is the corresponding desired output <br>
* $L$ denotes the number of layers <br>
* $a^L = a^L(x)$ is the vector activations output from the network when x is input

<h4>First Assumption</h4>

**The cost function can be written as an average $C = \frac{1}{n} \sum_x C_x$ over cost functions $C_x$ for individual training examples, $x$.**

For the quadratic cost function, a single training example is $C_x = \frac{1}{2} \|y-a^L \|^2$

Backpropagation computes the partial derivatives $\frac{\partial C_x}{\partial w}$ and $\frac{\partial C_x}{\partial b}$ at a single training example and recovering $\frac{\partial C}{\partial w}$ and $\frac{\partial C} {\partial b}$ by averaging over training examples.

<h4>Second Assumption</h4>

**The cost can be written as a function of the outputs from the neural network.**

![Network Output](imgs/cost_function_network.png)

Remember (in the case of the quadratic cost function) that a training example is fixed, so the output y is also fixed. So, the cost function is only a function of output activations. 

<h2>The Hadamard product</h2>

Suppose $s$ and $t$ are two vectors of the same dimension. Then we use $s \odot t$ to denote the elementwise product of the two vectors.

For example,

$$
\begin{eqnarray}
\left[\begin{array}{c} 1 \\ 2 \end{array}\right] 
  \odot \left[\begin{array}{c} 3 \\ 4\end{array} \right]
= \left[ \begin{array}{c} 1 * 3 \\ 2 * 4 \end{array} \right]
= \left[ \begin{array}{c} 3 \\ 8 \end{array} \right].
\tag{28}\end{eqnarray}
$$

<h2>The four fundamental equations behind backpropagation</h2>

We want to compute an error, $\delta^l_j$. With error, we can assume a network outputs $\sigma(z^l_j+\Delta z^l_j)$. The change propagates through later layers and causes an overall change of $\frac{\partial C}{\partial z^l_j} \Delta z^l_j$. 

So, we define error as,

$$
\begin{eqnarray} 
  \delta^l_j \equiv \frac{\partial C}{\partial z^l_j}.
\tag{29}\end{eqnarray}
$$

Backpropagation will give us a way of computing $\delta^l$ for every layer, and then relating those errors to the quantities of real interest, $\frac{\partial C}{\partial
w^l_{jk}}$ and $\frac{\partial C}{\partial b^l_j}$.

<h4>Equation One</h4>

An equation for the error in the output layer:

$$
\begin{eqnarray} 
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma'(z^L_j).
\tag{BP1}\end{eqnarray}
$$

The first term on the right, $\frac{\partial C}{\partial a^L_j}$, just measures how fast the cost is changing as a function of the jth output activation. The second term on the right, $\frac{\partial C}{\partial a^L_j}$, measures how fast the activation function $\sigma$ is changing at $z^L_j$.

And in matrix based form,

$$
\begin{eqnarray} 
  \delta^L = \nabla_a C \odot \sigma'(z^L).
\tag{BP1a}\end{eqnarray}
$$

Or,

$$
\begin{eqnarray} 
  \delta^L = (a^L-y) \odot \sigma'(z^L).
\tag{30}\end{eqnarray}
$$

<h4>Equation Two</h4>

An equation for the error $\delta^l$ in terms of the error in the next layer, $\delta^{l+1}$:

$$
\begin{eqnarray} 
  \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l),
\tag{BP2}\end{eqnarray}
$$